In [1]:
import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify, render_template, redirect


#################################################
# Database Setup
#################################################
engine = create_engine("sqlite:///../database/fifa_data.sqlite")

# Reflect an existing database into a new model
Base = automap_base()

# Reflect the tables
Base.prepare(engine, reflect=True)

# Save references to the tables
CountryData = Base.classes.country_data
MatchData = Base.classes.match_data
SquadData = Base.classes.squad_data

In [90]:
session = Session(engine)
squad_results = session.query(SquadData.country, SquadData.year, SquadData.age, SquadData.caps).all()


squad_data = []
for country, year, age, caps in squad_results:
    dict = {}
    dict["country"] = country
    dict["year"] = year
    dict["age"] = age
    dict["caps"] = caps
    squad_data.append(dict)

squad_data_pd = pd.DataFrame(squad_data)
squad_data_pd.caps = squad_data_pd.caps.str.extract('(\d+)').dropna().astype('int64')
squad_data_pd.dropna(inplace=True)

heat_data = squad_data_pd.groupby(['country', 'year']).mean()
heat_data.drop(axis = 1, columns = ['age'], inplace=True)
heat_data = heat_data.reset_index(level=0)
heat_data = heat_data.reset_index(level=0)
heat_data.rename(columns={"year": "x", "country": "y", "caps":"heat"}, inplace=True)


In [94]:
list(heat_data.heat)

[20.956521739130434,
 16.565217391304348,
 19.391304347826086,
 26.142857142857142,
 25.954545454545453,
 40.52173913043478,
 24.26086956521739,
 24.434782608695652,
 32.21739130434783,
 37.34782608695652,
 27.0,
 32.82608695652174,
 17.782608695652176,
 29.82608695652174,
 29.272727272727273,
 25.681818181818183,
 25.818181818181817,
 23.869565217391305,
 32.56521739130435,
 47.26086956521739,
 21.476190476190474,
 26.26086956521739,
 29.681818181818183,
 34.27272727272727,
 28.043478260869566,
 43.78260869565217,
 38.69565217391305,
 29.652173913043477,
 29.869565217391305,
 23.272727272727273,
 31.59090909090909,
 4.7894736842105265,
 17.454545454545453,
 28.47826086956522,
 29.608695652173914,
 30.217391304347824,
 26.954545454545453,
 24.652173913043477,
 34.95652173913044,
 42.17391304347826,
 20.15,
 23.863636363636363,
 28.26086956521739,
 31.0,
 39.91304347826087,
 39.04347826086956,
 34.04347826086956,
 51.34782608695652,
 19.636363636363637,
 27.565217391304348,
 32.73913043

In [77]:
squad_data_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4899 entries, 0 to 4911
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  4899 non-null   object 
 1   year     4899 non-null   int64  
 2   age      4899 non-null   int64  
 3   caps     4899 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 191.4+ KB
